### 3D UNet test
Trialing 3D UNet implementation found here: https://github.com/wolny/pytorch-3dunet

###Code to install and activate the correct conda environment 
conda install -c conda-forge mamba
mamba create -n pytorch3dunet -c pytorch -c nvidia -c conda-forge -c awolny pytorch-3dunet
conda activate pytorch3dunet

#### Running the model
__config files determine filepaths etc.__

train3dunet --config <CONFIG>



predict3dunet --config <CONFIG>


In [38]:
#create testing HDF5
%cd C:\Users\theev\OneDrive\Documents\LAB\Code\3DUNetTest
%ls

C:\Users\theev\OneDrive\Documents\LAB\Code\3DUNetTest
 Volume in drive C is OS
 Volume Serial Number is 9427-9109

 Directory of C:\Users\theev\OneDrive\Documents\LAB\Code\3DUNetTest

06/01/2023  01:32 PM    <DIR>          .
06/01/2023  01:32 PM    <DIR>          ..
06/01/2023  01:31 PM    <DIR>          Checkpoint
06/01/2023  01:32 PM             1,024 train.pytable
06/01/2023  01:32 PM             1,024 val.pytable
               2 File(s)          2,048 bytes
               3 Dir(s)  323,808,464,896 bytes free


In [18]:
%pip install tables


  Using cached tables-3.8.0-cp310-cp310-win_amd64.whl (3.6 MB)
  Using cached numpy-1.24.3-cp310-cp310-win_amd64.whl (14.8 MB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
  Using cached numexpr-2.8.4-cp310-cp310-win_amd64.whl (92 kB)
  Using cached blosc2-2.0.0-cp310-cp310-win_amd64.whl (2.0 MB)
  Using cached Cython-0.29.35-py2.py3-none-any.whl (988 kB)
  Using cached msgpack-1.0.5-cp310-cp310-win_amd64.whl (61 kB)
Note: you may need to restart the kernel to use updated packages.


In [87]:
import tables
import numpy as np
img_dtype = {}
img_dtype["raw"] = tables.Float64Atom()  
img_dtype["label"] = tables.UInt16Atom()  
filenameAtom = tables.StringAtom(itemsize=255) #hash

imgtypes = ["raw","label"]

#brute force two filepaths
training = [r"G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_133"]
# ,r"G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_077"

validating = [r"G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_133"]



In [90]:
raw_file.close()
val_file.close()

raw_file = tables.open_file(f"./train.h5", mode='w') 

block_shape={} 
patch_size = 400

reader.SetFileName(r"G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_133\UH_RectalCA_133_pre_ax_raw_label_resampled.mha")
reader.LoadPrivateTagsOn()
reader.ReadImageInformation()

size = reader.GetSize()
str_size = (size[2],size[1],size[0])
print(str_size)


block_shape["raw"]= np.array(str_size) #3 color channels
block_shape["label"]= np.array(str_size) #black and white



storage = {}
for imgtype in imgtypes: #init arrays for each storage
        storage[imgtype]= raw_file.create_earray(raw_file.root, imgtype, img_dtype[imgtype],  
                                                  shape=np.append([0],block_shape[imgtype]))


for file in training:
    for type in imgtypes:
        path = file + r"\UH_"+ file.rsplit("\\",1)[-1] + "_pre_ax_raw" + ("_label" if type == "label" else "") + "_resampled.mha"
        if path == r"G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_077\UH_RectalCA_077_pre_ax_raw_label_resampled.mha":
            path = r"G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_077\UH_RectalCA_077_pre_ax_label_resampled.mha"
        
        image = sitk.ReadImage(path, imageIO="MetaImageIO")
        n = [np.transpose(sitk.GetArrayFromImage(image[:,:,i])) for i in range(image.GetDepth())]
        print(np.array(n).shape)
        n = np.expand_dims(n,0)

        storage[type].append(n)

        print(path)
        print('done')

    




raw_file.close()




print("Doing validation now")

val_file = tables.open_file(f"./val.h5", mode='w')

storage = {}
for imgtype in imgtypes: #init arrays for each storage
        storage[imgtype]= val_file.create_earray(val_file.root, imgtype, img_dtype[imgtype],  
                                                  shape=np.append([0],block_shape[imgtype]))

for file in validating:
    for type in imgtypes:
        path = file + r"\UH_"+ file.rsplit("\\",1)[-1] + "_pre_ax_raw" + ("_label" if type == "label" else "") + "_resampled.mha"
        if path == r"G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_077\UH_RectalCA_077_pre_ax_raw_label_resampled.mha":
            path = r"G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_077\UH_RectalCA_077_pre_ax_label_resampled.mha"
        
        image = sitk.ReadImage(path, imageIO="MetaImageIO")
        n = [np.transpose(sitk.GetArrayFromImage(image[:,:,i])) for i in range(image.GetDepth())]
        print(np.array(n).shape)
        n = np.expand_dims(n,0)

        storage[type].append(n)

        print(path)
        print('done')

val_file.close()


(192, 180, 180)
(192, 180, 180)
G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_133\UH_RectalCA_133_pre_ax_raw_resampled.mha
done
(192, 180, 180)
G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_133\UH_RectalCA_133_pre_ax_raw_label_resampled.mha
done
Doing validation now
(192, 180, 180)
G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_133\UH_RectalCA_133_pre_ax_raw_resampled.mha
done
(192, 180, 180)
G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_133\UH_RectalCA_133_pre_ax_raw_label_resampled.mha
done


In [61]:
file = r"G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_133\UH_RectalCA_133_pre_ax_raw_resampled.mha"
image = sitk.ReadImage(file, imageIO="MetaImageIO")
print(image.GetPixelIDTypeAsString()) 

file = r"G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_133\UH_RectalCA_133_pre_ax_raw_label_resampled.mha"
image = sitk.ReadImage(file, imageIO="MetaImageIO")
print(image.GetPixelIDTypeAsString()) 

64-bit float
16-bit unsigned integer
